# Project Title
### WORLD CITIES LAND TEMPERATURE ETL & DATA WAREHOUSE

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import os
import logging
import boto3
from botocore.exceptions import ClientError
import psycopg2
from helpers.sql import sql_create_tables, staging_insert, sql_drop_tables, sql_insert_tables
from configparser import ConfigParser

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [19]:
# Read in the data
directory = os.path.realpath("..") + '/global-land-temp-dw/'
tables = ['GlobalLandTemperaturesByMajorCity.csv', 'GlobalLandTemperaturesByCity.csv']

df_major_city = pd.read_csv(directory + "data/" + tables[0])
df_city = pd.read_csv(directory + "data/" + tables[1])

In [ ]:
# Drop NA values This data requires all values to be available

def drop_na(pd_df):

    print(f'Number of rows before removing NA: {pd_df.shape[0]:,}')

    pd_df.dropna(axis=0, inplace=True)

    print(f'Number of rows after removing NA: {pd_df.shape[0]:,}')


drop_na(df_major_city)
drop_na(df_city)


In [ ]:
# Add column for Boolean Major city
df_major_city = df_major_city.assign(major_city='true')
df_city = df_city.assign(major_city='false')



In [ ]:
# Concatnate both datasets
dframes = [df_major_city, df_city]
df = pd.concat(dframes, ignore_index=True)

In [ ]:
df.head()

In [ ]:
# Sort data by year
df.sort_values(by='dt',inplace=True)

In [ ]:
df.head(1000)


In [ ]:
# Confirm Data types are as expected
df.dtypes

In [ ]:
# Save data to CSV to load into SQL redshift
df.to_csv(directory + "data/dataset.csv")

In [ ]:
# Upload to s3 Function from AWS Services https://boto3.amazonaws.com/v1/documentation/api/1.10.46/guide/s3-uploading-files.html

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
# Get Bucket Name from configurations 
config = ConfigParser()
config.read('DW.cfg')
file_log = config.get('S3', 'log_data')

upload_file(directory + "data/dataset.csv", file_log, "logs/log_data.csv")


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Star Schema

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [2]:
# Get Database Variables
config = ConfigParser()
config.read('DW.cfg')

print("{}, {}, {}, {}, {}".format(*config['CLUSTER'].values()))


# Connect to redshift database
conn = psycopg2.connect("""host={} 
                           dbname={} 
                           user={} 
                           password={}
                           port={}"""\
                           .format(*config['CLUSTER'].values()    
))

# conn.set_session(autocommit=True)
cur = conn.cursor()


dwhcluster.cedlo7g6palf.eu-west-1.redshift.amazonaws.com, landtempdb, dwhuser, Passw0rd, 5439


In [3]:
# Drop all table if exists
for query in sql_drop_tables:
    cur.execute(query)
    conn.commit()

In [4]:
# Create tables
for query in sql_create_tables:
    cur.execute(query)
    conn.commit()


In [5]:
# Copy data to Staging
s3_bucket = "s3://{}/logs/".format(config.get('S3','log_data'))

sql_query = staging_insert.format(
    s3_bucket,
    config.get('IAM_ROLE','ARN'),
    config.get('CLUSTER','dwh_region')
    )
# print(sql_query)
cur.execute(sql_query)
conn.commit()


    COPY staging_events (
        index,
        dt,
        AverageTemperature,
        AverageTemperatureUncertainty,
        City,
        Country,
        Latitude,
        Longitude,
        major_city
    )
    FROM  's3://global-land-temp/logs/'
    CREDENTIALS 'aws_iam_role=arn:aws:iam::879294216748:role/dwhRole'
    REGION 'eu-west-1'
    DELIMITER ','
    DATEFORMAT 'auto'
    IGNOREHEADER 1
    ;



In [6]:
# Insert data from staging to facts and dimensions
for query in sql_insert_tables:
    # print(query)
    cur.execute(query)
    conn.commit()

In [ ]:
# Write code here

In [ ]:
# Close Connection to the DB
conn.close()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [9]:
# Perform quality checks here
def data_quality_check(cur, table):
    """
    Function that check if table has rows

    Argument:
    cur: <object> Posgres connection cursor
    table: <list> list of tables from the schema

    returns None
    
    """
    sql_test = """
        SELECT COUNT(*) 
        FROM {}
        LIMIT 10;
        """.format(table)
    try:
        cur.execute(sql_test)
        data_test = cur.fetchall()

        if len(data_test) < 1:
            raise ValueError("Table '{}' is empty".format(table))

        if data_test[0][0] < 1:
            raise ValueError("Table '{}' has no rows".format(table))

        print('Data Quality SUCCESS for table: {}. Total rows: {}'\
                .format(table, data_test[0][0]))



    except psycopg2.errors.UndefinedTable as e:
        # Close Connection
        conn.close()
        raise Exception(e)

    


In [10]:
tables = ['readings_by_city', 'cities', 'time']

for table in tables:

    data_quality_check(cur, table)

Data Quality SUCCESS for table: readings_by_city. Total rows: 8609236
Data Quality SUCCESS for table: cities. Total rows: 3610
Data Quality SUCCESS for table: time. Total rows: 3167


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [16]:
data_dictionary = [
    {   
        'table':'Dimension cities',
        'field name': 'city_id',
        'data type': 'BIGINT IDENTITY(0,1)',
        'data format': 'NNNNNNN',
        'field size': 'VARIABLE',
        'description':'Primary Key Index number for each row',
        'example':'03'
    },
    {
        'table':'Dimension cities',
        'field name': 'city',
        'data type': 'TEXT',
        'data format': '',
        'field size': 'VARIABLE',
        'description':'city name, comes from staging_events',
        'example':'Barcelona'
    },
    {
        'table':'Dimension cities',
        'field name': 'country',
        'data type': 'TEXT',
        'data format': '',
        'field size': 'VARIABLE',
        'description':'country name, comes from staging_events',
        'example':'Venezuela'
    },
    {
        'table':'Dimension cities',
        'field name': 'latitude',
        'data type': 'TEXT',
        'data format': 'NN.NT',
        'field size': 'VARIABLE',
        'description':'city latitude, comes from staging_events',
        'example':'12.5N'
    },
    {
        'table':'Dimension cities',
        'field name': 'longitude',
        'data type': 'TEXT',
        'data format': 'NN.NT',
        'field size': 'VARIABLE',
        'description':'city longitude, comes from staging_events',
        'example':'12.5E'
    },
    {
        'table':'Dimension cities',
        'field name': 'major_city',
        'data type': 'BOOLEAN',
        'data format': 'true/false',
        'field size': '5',
        'description':'Boolean that identifies if city is major or not from staging_events',
        'example':'true'
    },
    {
        'table':'Dimension time',
        'field name': 'dt',
        'data type': 'DATE',
        'data format': 'YYYY-MM-DD',
        'field size': '10',
        'description':'date of temperature measure, comes from staging_events',
        'example':'2018-02-07'
    },
    {
        'table':'Dimension time',
        'field name': 'day',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'day of temperature measure, comes from column dt',
        'example':'07'
    },
    {
        'table':'Dimension time',
        'field name': 'month',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'month of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'week',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'week of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'weekday',
        'data type': 'SMALLINT',
        'data format': 'NN',
        'field size': '2',
        'description':'week day number of temperature measure, comes from column dt',
        'example':'02'
    },
    {
        'table':'Dimension time',
        'field name': 'year',
        'data type': 'SMALLINT',
        'data format': 'NNNN',
        'field size': '4',
        'description':'year of temperature measure, comes from column dt',
        'example':'2018'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'by_city_id',
        'data type': 'BIGINT IDENTITY(0,1)',
        'data format': 'NNNNNN',
        'field size': 'VARIABLE',
        'description':'Primary Key Index number for each row',
        'example':'2'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'city_id',
        'data type': 'NUMERIC',
        'data format': 'NNNNNN',
        'field size': 'VARIABLE',
        'description':'Foreign Key Index number from cities table',
        'example':'4525'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'avg_temp',
        'data type': 'NUMERIC(7,3)',
        'data format': 'NNNN.NNN',
        'field size': '7',
        'description':'temperature reading comes from staging_events',
        'example':'25.025'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'avg_temp_uncertainty',
        'data type': 'NUMERIC(7,3)',
        'data format': 'NNNN.NNN',
        'field size': '7',
        'description':'temperature uncertainty reading comes from staging_events',
        'example':'1.025'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'date',
        'data type': 'DATE',
        'data format': 'YYYY-MM-DD',
        'field size': '7',
        'description':'date of temperature measure, comes from staging_events',
        'example':'2018-02-07'
    },
    {
        'table':'Fact readings_by_city',
        'field name': 'major_city',
        'data type': 'BOLEAN',
        'data format': '',
        'field size': '5',
        'description':'Boolean that identifies if city is major or not from staging_events',
        'example':'true'
    },
    
    
       
]

In [21]:
data_dic_df = pd.DataFrame(data_dictionary)

data_dic_df.to_csv(directory+"data_dict.csv")

data_dic_df


,table,field name,data type,data format,field size,description,example
0,Dimension cities,city_id,"BIGINT IDENTITY(0,1)",NNNNNNN,VARIABLE,Primary Key Index number for each row,03
1,Dimension cities,city,TEXT,,VARIABLE,"city name, comes from staging_events",Barcelona
2,Dimension cities,country,TEXT,,VARIABLE,"country name, comes from staging_events",Venezuela
3,Dimension cities,latitude,TEXT,NN.NT,VARIABLE,"city latitude, comes from staging_events",12.5N
4,Dimension cities,longitude,TEXT,NN.NT,VARIABLE,"city longitude, comes from staging_events",12.5E
5,Dimension cities,major_city,BOOLEAN,true/false,5,Boolean that identifies if city is major or no...,true
6,Dimension time,dt,DATE,YYYY-MM-DD,10,"date of temperature measure, comes from stagin...",2018-02-07
7,Dimension time,day,SMALLINT,NN,2,"day of temperature measure, comes from column dt",07
8,Dimension time,month,SMALLINT,NN,2,"month of temperature measure, comes from colum...",02
9,Dimension time,week,SMALLINT,NN,2,"week of temperature measure, comes from column dt",02


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.